In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [5]:
!ls

Aleksey_Savrasov  Da_Vinci	 preprocessing.ipynb  transfer_resnet50.ipynb
Claude_Monet	  Frida_Kahlo	 Rembrandt	      Van_Gogh
code		  Michelangelo	 resnet50+dropout.h5  Wassily_Kandisky
data.h5		  Pablo_Picasso  resnet50.h5
data_x.h5	  Paul_Cezanne	 resnet50.ipynb
data_x.pickle	  Pierre_Renoir  Salvador_Dali


In [0]:
import os
os.chdir('drive/nnfl/NNFL/')

In [0]:
import numpy as np
import keras
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,Dropout, Reshape
from keras.models import Model, load_model,Sequential

In [9]:
resnet = keras.applications.resnet50.ResNet50(include_top=False, weights='imagenet' ,input_shape=(224,224,3))


94658560/94653016 [==============================] - 9s 0us/step


In [36]:
X_in = Input(shape=(224,224,3))
X = resnet(X_in)
X = Flatten()(X)
X = Dropout(0.5)(X)
X = Dense(12,activation='softmax')(X)

model = Model(inputs=X_in,outputs=X)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
resnet50 (Model)             (None, 1, 1, 2048)        23587712  
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 12)                24588     
Total params: 23,612,300
Trainable params: 23,559,180
Non-trainable params: 53,120
_________________________________________________________________


In [0]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
import h5py
hf = h5py.File('data_x.h5','r')
x = hf.get('dataset_1')
y = hf.get('dataset_2')
print(x.shape,y.shape)

(2997, 224, 224, 3) (2997, 12, 1)


In [0]:
p=np.random.permutation(x.shape[0])
x = np.asarray(x)
y = np.asarray(y)
y = y.reshape((2997,12))
x = x[p]
y = y[p]

x_train = x[:2698]
y_train = y[:2698]
x_test = x[2698:]
y_test = y[2698:]

In [38]:
model.fit(x_train, y_train,validation_data=(x_test,y_test), epochs = 15, batch_size = 64)

Train on 2698 samples, validate on 299 samples
Epoch 1/15
2698/2698 [==============================] - 113s 42ms/step - loss: 2.1215 - acc: 0.3999 - val_loss: 5.1231 - val_acc: 0.0936
Epoch 2/15
2698/2698 [==============================] - 74s 28ms/step - loss: 1.9993 - acc: 0.4162 - val_loss: 13.0252 - val_acc: 0.1037
Epoch 3/15
2698/2698 [==============================] - 74s 28ms/step - loss: 1.8981 - acc: 0.4248 - val_loss: 10.8528 - val_acc: 0.1338
Epoch 4/15
2698/2698 [==============================] - 74s 27ms/step - loss: 1.5644 - acc: 0.4959 - val_loss: 10.3455 - val_acc: 0.1538
Epoch 5/15
2698/2698 [==============================] - 74s 27ms/step - loss: 1.4517 - acc: 0.5408 - val_loss: 4.0028 - val_acc: 0.2809
Epoch 6/15
2698/2698 [==============================] - 74s 27ms/step - loss: 1.2741 - acc: 0.5938 - val_loss: 2.1577 - val_acc: 0.4247
Epoch 7/15
2698/2698 [==============================] - 74s 27ms/step - loss: 1.1055 - acc: 0.6245 - val_loss: 3.4735 - val_acc: 0.39

In [0]:
model.save('tranfer_resnet.h5')